<a href="https://colab.research.google.com/github/TajikDanyal/Pothole-Detection/blob/master/BRATS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!wget https://www.cbica.upenn.edu/sbia/Spyridon.Bakas/MICCAI_BraTS/2019/MICCAI_BraTS_2019_Data_Training.zip

--2019-09-12 08:36:11--  https://www.cbica.upenn.edu/sbia/Spyridon.Bakas/MICCAI_BraTS/2019/MICCAI_BraTS_2019_Data_Training.zip
Resolving www.cbica.upenn.edu (www.cbica.upenn.edu)... 165.123.244.124
Connecting to www.cbica.upenn.edu (www.cbica.upenn.edu)|165.123.244.124|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2759083030 (2.6G) [application/zip]
Saving to: ‘MICCAI_BraTS_2019_Data_Training.zip’

MICCAI_BraTS_2019_D 100%[===================>]   2.57G  34.7MB/s    in 79s     

2019-09-12 08:37:31 (33.1 MB/s) - ‘MICCAI_BraTS_2019_Data_Training.zip’ saved [2759083030/2759083030]



In [0]:
from zipfile import ZipFile
zip = ZipFile('MICCAI_BraTS_2019_Data_Training.zip')
zip.extractall()

In [0]:
import os
import numpy as np
import nibabel as nib
import random
import matplotlib.pyplot as plt
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras import backend as keras
from keras.utils import Sequence

In [0]:
PATH = "/content/MICCAI_BraTS_2019_Data_Training/HGG/"

In [0]:
class MyGenerator(Sequence):
    def __init__(self, Path, Type="Train", Ratio=0.8):
        self.Path=Path
        self.Folders=os.listdir(Path)
        self.Type = Type
        self.Train, self.Test, self.Valid = Split(self.Folders, Ratio)
    def __len__(self):
        if self.Type is "Train" :
            return len(self.Train)
        elif self.Type is "Test":
            return len(self.Test)
        return len(self.Valid)

    def __getitem__(self, idx):
        if self.Type is "Train" :
            return ReadImageDataFromFolderFiles(self.Train[idx])
        elif self.Type is "Test" :
            return ReadImageDataFromFolderFiles(self.Test[idx])
        return ReadImageDataFromFolderFiles(self.Valid[idx])
         

In [0]:
def ReadDataset(Path):
    return os.listdir(Path)

In [0]:
def SetLabel(FileName):
    if"t1." in FileName :
        return "T1"
    elif "t1ce." in FileName:
        return "T1CE"
    elif "seg." in FileName:
        return "Segmented"
    elif "flair." in FileName:
        return "Flair"
    elif "t2." in FileName:
        return "T2"
    return "Undefined"

In [0]:
def ReadImageDataFromFolderFiles(FolderName):
    Files = os.listdir(PATH+FolderName)
    Files.sort()
    Images = [nib.load( PATH+FolderName+"/"+File ).get_fdata() for File in Files]
    XData = np.zeros( (Images[0].shape[2], Images[0].shape[0], Images[0].shape[1], 4) )
    YData = np.zeros( (Images[0].shape[2], Images[0].shape[0], Images[0].shape[1], 1) )
    for i in range(Images[0].shape[2]):
        XData[i,:,:,0] = Images[0][:,:,i]
        XData[i,:,:,1] = Images[2][:,:,i]
        XData[i,:,:,2] = Images[3][:,:,i]
        XData[i,:,:,3] = Images[4][:,:,i]
        YData[i,:,:,0] = Images[1][:,:,i]
    return XData, YData

In [0]:
def DisplaySample(FolderName, index):
    print("****IN: "+str(index+1)+" PATH="+PATH+FolderName+"****")
    Files = os.listdir(PATH+FolderName)
    fig, axes = plt.subplots(1, len(Files))
    count=0
    TempFiles=Files
    TempFiles.sort()
    print(TempFiles)
    for File in Files:
        img = nib.load(PATH+FolderName+"/"+File)
        img_data = img.get_fdata()
        #raise ValueError(img_data.shape)
        axes[count].title.set_text(SetLabel(File))
        axes[count].imshow(img_data[:,:,77].T, aspect="equal",  interpolation='nearest', origin="lower")
        count+=1
    plt.show()

In [0]:
def AnimateSample(FolderName, index):
    print("****IN: "+str(index+1)+" PATH="+PATH+FolderName+"****")
    Files = os.listdir(PATH+FolderName)
    TempFiles=Files
    TempFiles.sort()
    for File in Files:
        img = nib.load(PATH+FolderName+"/"+File)
        img_data = img.get_fdata()
        #raise ValueError()
        
        for i in range(img.shape[2]):
            plt.imshow(img_data[:,:,i].T)
            plt.title(File)
            plt.show()


In [0]:
def Split(Folders, Ratio=0.8):
    random.shuffle(Folders)
    Train = []
    Test = []
    Valid = []
    SplitPoint = int(Ratio*len(Folders))
    Train = Folders[0:SplitPoint]
    Test = Folders[SplitPoint:len(Folders)]
    SplitPoint = int(Ratio*len(Train))
    Valid = Train[SplitPoint:len(Train)]
    Train = Train[0:SplitPoint]
    return Train, Test, Valid

In [0]:
def PreProcessAndPrepareDataset(Path):
    Folders=ReadDataset(Path)
    X = np.zeros((len(Folders),155*5,240,240))
    for Folder in Folders:
        ImageList = ReadImageDataFromFolderFiles(Folder)
        ImageArray = np.asarray(ImageList)
        print(ImageArray.shape)
        ImageArray = ImageArray.reshape((155*5,240,240))
        raise ValueError(ImageArray.shape)

In [0]:
def unet(pretrained_weights = None,input_size = (240, 240,4)):
    inputs = Input(input_size)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
    drop5 = Dropout(0.5)(conv5)

    up6 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
    merge6 = concatenate([drop4,up6], axis = 3)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

    up7 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
    merge7 = concatenate([conv3,up7], axis = 3)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

    up8 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
    merge8 = concatenate([conv2,up8], axis = 3)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

    up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
    merge9 = concatenate([conv1,up9], axis = 3)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv9 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv10 = Conv2D(1, 1, activation = 'sigmoid')(conv9)

    model = Model(input = inputs, output = conv10)

    model.compile(optimizer = Adam(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['accuracy'])

        
    return model

In [0]:
Folders = ReadDataset(PATH)
#TrainGenerator = MyGenerator(PATH)
#Model=unet()
#Model.fit_generator(TrainGenerator, epochs=2, steps_per_epoch=259)
for Index, Folder in enumerate(Folders):
    AnimateSample(Folders[0], 0)
#Train, Test, Valid = Split(Folders, 0.8)